In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import pyfolio as pf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import yfinance as yf

# Define the stock symbol and date range
ticker = "TSLA"
start_date = "2015-01-01"
end_date = "2025-01-31"

# Fetch historical stock data
tsla_data = yf.download(ticker, start=start_date, end=end_date)

# Display the first few rows
print(tsla_data.head())

# Save data to CSV (Optional)
tsla_data.to_csv("/content/drive/MyDrive/Portfolio Management Optimization/data/TSLA_stock_data.csv")
print("Data saved to TSLA_stock_data.csv")

[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open    Volume
Ticker           TSLA       TSLA       TSLA       TSLA      TSLA
Date                                                            
2015-01-02  14.620667  14.883333  14.217333  14.858000  71466000
2015-01-05  14.006000  14.433333  13.810667  14.303333  80527500
2015-01-06  14.085333  14.280000  13.614000  14.004000  93928500
2015-01-07  14.063333  14.318667  13.985333  14.223333  44526000
2015-01-08  14.041333  14.253333  14.000667  14.187333  51637500
Data saved to TSLA_stock_data.csv


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Portfolio Management Optimization/data/TSLA_stock_data.csv")
data.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
1,Date,NaN,NaN,NaN,NaN,NaN
2,2015-01-02,14.620667457580566,14.883333206176758,14.21733283996582,14.857999801635742,71466000
3,2015-01-05,14.005999565124512,14.433333396911621,13.810667037963867,14.303333282470703,80527500
4,2015-01-06,14.085332870483398,14.279999732971191,13.61400032043457,14.003999710083008,93928500
